### 1. Installing the required dependencies

In [ ]:
# Installing Tensorflow gpu
!pip install tensorflow-gpu

In [ ]:
# Verification of the Tensorflow installation
!python -c "import tensorflow as tf;print(tf.reduce_sum(tf.random.normal([1000, 1000])))"

In [ ]:
# Installing Opencv library
!pip install opencv-python

In [ ]:
# Installing Coco api
!pip install cython

In [ ]:
import os

In [ ]:
# Creating different folders for data support

if not os.path.exists('Tensorflow'):
    !mkdir Tensorflow\models
    !mkdir Tensorflow\labelimg
    !mkdir Tensorflow\workspace 
    !mkdir Tensorflow\workspace\annotations
    !mkdir Tensorflow\workspace\images
    !mkdir Tensorflow\workspace\models
    !mkdir Tensorflow\workspace\pre-trained-models
    !mkdir Tensorflow\workspace\images\train
    !mkdir Tensorflow\workspace\images\test
    !mkdir Tensorflow\scripts\preprocessing

In [ ]:
# Git cloning the entire Tensorflow object Detection repositories
!cd Tensorflow/&& git clone https://github.com/tensorflow/models

In [ ]:
# This is to check the protobuf installation
# Navigate in cmd to TensorFlow/models/research/
protoc object_detection/protos/*.proto --python_out=. # Run in command prompt

# Or run directly here
!cd Tensorflow/models/research/ && protoc object_detection/protos/*.proto --python_out=.

In [ ]:
# Protobuf verifications
# Navigate in cmd to  TensorFlow/models/research/
# Run in command prompt 
for /f %i in ('dir /b object_detection\protos\*.proto') do protoc object_detection\protos\%i --python_out=.    

# Or run directly here
!cd Tensorflow/models/research/ && for /f %i in ('dir /b object_detection\protos\*.proto') do protoc object_detection\protos\%i --python_out=.

In [ ]:
# Installing the Object Detection API
!cd Tensorflow/models/research && cp object_detection/packages/tf2/setup.py . && python -m pip install --use-feature=2020-resolver .

In [ ]:
# Verifying the installation
# From within TensorFlow/models/research/
!cd TensorFlow/models/research/object_detection/builders/ && python model_builder_tf2_test.py


In [ ]:
!pip install --upgrade pyqt5 lxml

In [ ]:
# Installing the GUI for Image Labelling to create xml file for each image
!pip install labelImg

### 2. Collecting Images to train the model

In [ ]:
labels = ['thumbsup', 'thumbsdown', 'palm', 'fist','ok']
number_of_images = 100 

In [ ]:
Images_location = os.path.join('Tensorflow', 'workspace', 'data')

In [ ]:
for label in labels:
    path = os.path.join(Images_location, label)
    if not os.path.exists(path):
        !mkdir {path}

In [ ]:
# *** Dont run this cell unless you need to capture live images for train data set ***

# Collecting images using webcam

counter = 0
for label in labels:
    for image_no in range(number_of_images):
        cam = cv2.VideoCapture(0)
        cam.set(3, 640)
        cam.set(4, 480)
        cam.set(10, 100)
        ret, frame = cam.read()
        cv2.imshow("Video", frame)
        img_location = os.path.join(Images_location, label, f"{label}_{counter}.jpg")
        cv2.imwrite(img_location, frame)  # Saving the  image
        print(f"{label}_{counter}.jpg")
        counter += 1
        time.sleep(5)
        if cv2.waitKey(5) % 256 == 27:
            print('Escape pressed. Closing the application')
            break

        cam.release()
        cv2.destroyAllWindows()

In [ ]:
# Run the GUI for image labelling
labelimage_path = os.path.join('Tensorflow', 'labelimg')
!cd {labelimage_path} && python labelImg.py

Save the xml files of train and test images in their respective folders

Split the total images in the ratio of 9:1 and paste into train and test folders 

## 3. Creating Label Map

In [ ]:
labels = [{'name':'thumbsup', 'id':1}, {'name':'thumbsdown', 'id':2}, {'name':'palm', 'id':3}, {'name':'fist', 'id':4}, {'name':'ok', 'id':5}]
label_map_name = 'label_map.pbtxt'
label_path = os.path.join('Tensorflow','workspace','annotations',label_map_name)


In [ ]:
with open(label_path, 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [ ]:
scripts_path = os.path.join('Tensorflow','scripts','preprocessing','generate_tfrecord.py')

In [ ]:
# Generating the *.record file for test data and train data after mapping every image for its respective xml file
!cd Tensorflow/scripts/ && python generate_tfrecord.py -x D:work/Tensorflow/workspace/images/train -l D:/work/Tensorflow/workspace/annotations/label_map.pbtxt -o D:/work/Tensorflow/workspace/annotations/train.record

!cd Tensorflow/scripts/ && python generate_tfrecord.py -x D:work/Tensorflow/workspace/images/test -l D:/work/Tensorflow/workspace/annotations/label_map.pbtxt -o D:/work/Tensorflow/workspace/annotations/test.record


## 4. Downloading the model from Tensorflow model garden

In [ ]:
model_name = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
model_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

In [ ]:
!pip install wget

In [ ]:
import wget

In [ ]:
!cd Tensorflow/workspace/pre-trained-models/
wget.download(model_url)

In [ ]:
!move ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8*.gz Tensorflow/workspace/pre-trained-models


Extract the tar file and paste the contents in the pre-trained-models folder

In [ ]:
# Creating folder for OUR CUSTOM model


!mkdir Tensorflow\workspace\models\my_ssd_mobnet

In [ ]:
# Copy the 'pipeline.config' file from pre-trained-models folder to  'my_ssd_resnet50_v1_fpn' folder


!copy Tensorflow\workspace\pre-trained-models\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\pipeline.config Tensorflow\workspace\models\my_ssd_mobnet\

In [ ]:
import object_detection

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
#Pipe line config path 
pipeline_path = os.path.join('Tensorflow','workspace','models', 'my_ssd_mobnet', 'pipeline.config')

In [ ]:
#configuration_path = os.path(Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config)
config = config_util.get_configs_from_pipeline_file(pipeline_path)  
config

### Open the pipeline.config file and make the following changes 



*** Under ssd ***

* num_classes: 5              ##### Number of items to detect ######




*** Under fine_tune_checkpoint ***

* fine_tune_checkpoint: "Tensorflow\\workspace\\pre-trained-models\\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-                                         8\\checkpoint\\ckpt-0"                       #### Path to checkpoint of pre-trained model######

* fine_tune_checkpoint_type: "detection" #####  Set this to "detection" since we want to be training the full detection model                                                    ######

* fine_tune_checkpoint: "Tensorflow\\workspace\\pre-trained-models\\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-                   8\\checkpoint\\ckpt-0"                                          ##### Path to checkpoint of pre-trained model######

* num_steps: 50000                                ######  Based on your config  ###### i used 2000 and 2500

* fine_tune_checkpoint_type: "detection"       #####  Set this to "detection" since we want to be training the full detection model                                                    ######




*** Under train_input_reader  ***

* label_map_path: "Tensorflow\\workspace\\annotations\\label_map.pbtxt"         ##### Path to label map  #####
    
* input_path: "Tensorflow\\workspace\\annotations\\train.record"              ##### Path to training record file ####
  
* metrics_set: "coco_detection_metrics"                                         #### Since the model is coco based  ####




*** Under eval_input_reader ***

* label_map_path: "Tensorflow\\workspace\\annotations\\label_map.pbtxt"         #### Path to label map file #####
* input_path: "Tensorflow\\workspace\\annotations\\test.record"               ###### Path to test record file  #####
  
    
    
    
    
    
    
    
    

In [ ]:
# Runing config again to see if the changes we made is visible or not
config

## 5. Training the model

 copy the TensorFlow/models/research/object_detection/model_main_tf2.py into workspace folder

In [ ]:
# To check if your machine has GPU installed. Install CUDA and cuDNN for your respective TF version(kindly refer instructions)


physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num of GPU:", len(physical_devices))

In [ ]:
# Training the model on our data set
# Run the command in cmd to see the progress
# add '--num_train_steps=2500' value of your choice for the compuatation, else the model will run for 50000 steps


python Tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=Tensorflow\workspace\models\my_ssd_mobnet --pipeline_config_path=Tensorflow\workspace\models\my_ssd_mobnet\pipeline.config 

In [ ]:
# Run Tensorboard to see the losses in the model
# Navigate to Tensorflow/workspace and run the following command in cmd

tensorboard --logdir=models/my_ssd_mobnet


# Once you receive a url for the port(ex, http://localhost:6006/), paste it in browser to see the results.



## 6. Load model from auto saved checkpoint

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(pipeline_path)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)




# Restore checkpoint from the latest check point 3(Always use the last checkpoint)
checkpoint_path = os.path.join('Tensorflow','workspace','models', 'my_ssd_mobnet')
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(checkpoint_path, 'ckpt-3')).expect_partial()



@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

## 7. Object Detection from Images

In [ ]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
labelmap_path = os.path.join('Tensorflow', 'workspace','annotations','label_map.pbtxt')
category_index = label_map_util.create_category_index_from_labelmap(label_map_path=labelmap_path)
IMAGE_PATH = os.path.join('Tensorflow', 'workspace','images','test',
                          'ok.9533399a-6d8e-11ec-a41a-98fa9b97e1dd.jpg')


In [ ]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.8,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

## 8. Detect from Webcam

In [ ]:
import cv2
import numpy as np
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections
    
    
    
    

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break